In [ ]:
import os
from pathlib import Path
from boutdata import collect
from matplotlib import pyplot as plt
import numpy as np
from netCDF4 import Dataset
import glob
from scipy.ndimage import zoom

In [ ]:

s1 = r"/Volumes/TOSHIBA EXT/Deep Learning/Data/Probe"
s2 = r"/Volumes/TOSHIBA EXT/Deep Learning/Data/Simulation"
e = r"data_numpy"

print(Path("/Volumes/TOSHIBA EXT/Deep Learning/Data/Probe").exists())


DATA_LOCATION_PROBE = Path(s1)
DATA_LOCATION_SIMULATION = Path(s2)
EXPORT_LOCATION = Path(e)

field_list = ['n']
probe_list = ['3']


In [ ]:
probe_data = {}
for p in probe_list:
    probe_data[p] = {}

    probe_vars = [f"{f}{p}" for f in field_list]
    probe_vars.append('t_array')
    for var in probe_vars:
        for file in glob.glob(f'{DATA_LOCATION_PROBE}/BOUT.fast.*'):
            try:
                with Dataset(file) as f:
                    probe_data[p][var] = f[var][:]
                    print(f"data {var} loaded from {file.split('/')[-1]}")
                    break
            except:
                continue

print(probe_data)

In [ ]:
nx, nz = collect('nx', path=DATA_LOCATION_SIMULATION), collect('nz', path=DATA_LOCATION_SIMULATION)

probe_pos = {}
for p in probe_list:
    probe_pos[p] = {}
    frac = 0.1*(int(p)+2)
    probe_pos[p]['x'] = round(frac*int(nx))
    probe_pos[p]['z'] = round(0.5*int(nz))

print(probe_pos)

In [ ]:
sim_data = {}
sim_data['t_array'] = collect('t_array', path = DATA_LOCATION_SIMULATION)
for f in field_list:
    sim_data[f] = collect(f, path = DATA_LOCATION_SIMULATION)

In [ ]:
zoom_t_start = 30000 
zoom_t_end = 35000
zoom_y_min = 0.0
zoom_y_max = 0.9

for p in probe_list:
    for f in field_list:
        fig, ax = plt.subplots()
    
        ax.plot(probe_data[p]['t_array'], probe_data[p][f"{f}{p}"], label = 'probe data')
        ax.plot(sim_data['t_array'], sim_data[f][:, probe_pos[p]['x'], 0, probe_pos[p]['z']], "x", label = 'simulation data')
        ax.legend(loc="upper left")
    
        ax_inset = ax.inset_axes([1.1, 0.1, 0.8, 0.8])
    
        ax_inset.plot(probe_data[p]['t_array'], probe_data[p][f"{f}{p}"], label = 'probe data')
        ax_inset.plot(sim_data['t_array'], sim_data[f][:, probe_pos[p]['x'], 0, probe_pos[p]['z']], "x", label = 'simulation data')
    
        ax_inset.set_xlim(zoom_t_start, zoom_t_end)
        ax_inset.set_ylim(zoom_y_min, zoom_y_max)
        
        ax.indicate_inset_zoom(ax_inset, edgecolor="black")
    
        fig.suptitle(f"probe: {p}, field: {f}")

In [ ]:
# Download the relevant simulation data as npy files

dens_array = np.array(sim_data['t_array'])
print(dens_array.ndim)
valid_images = dens_array[23:,:,:]
dim = 256
scale_factors = (1, dim/sim_data.shape[1], dim/sim_data.shape[2])
images = zoom(valid_images, scale_factors, order=1)  # order=1 is bilinear

print(np.shape(images))

plt.figure()
plt.imshow(images[0,:,:].T, origin='lower', cmap='viridis')
plt.colorbar(label="n (arb. units)")
plt.xlabel("x index")
plt.ylabel("z index")
plt.title("Density at timestep 50")
plt.grid(False)
plt.show()
